## SVD
基于矩阵分解的推荐。  
首先我们需要明确所要解决的问题，即对于一个M行（M个item），N列（N个user）的矩阵，当然这个矩阵是很稀疏的，即用户对于项目的评分是不充分的，大部分是没有记录的，我们的任务是要通过分析已有的数据（观测数据）来对未知数据进行预测，即这是一个矩阵补全（填充）任务。矩阵填充任务可以通过矩阵分解技术来实现。  
### 1 Traditional SVD:
当然人们首先想到的矩阵分解技术是SVD（奇异值）分解，在这我命名为traditional SVD（传统并经典着），直接上公式：
$$
M_{m\times{n}}=U_{m\times{k}}\sum_{k\times{k}}V^T_{k\times{n}}\\
k<=rank(M),\\
当k=rank(M)时称为紧奇异值分解（无损），否则称为截断奇异值分解（有损）
$$
如果想运用SVD分解的话，有一个前提是要求矩阵是稠密的，即矩阵里的元素要非空，否则就不能运用SVD分解。很显然我们的任务还不能用SVD，所以一般的做法是先用均值或者其他统计学方法来填充矩阵，然后再运用SVD分解降维。由于在实际过程中，元素缺失值是非常多的，这就导致了早期的 SVD 不论通过以上哪种方法进行补全在实际的应用之中都是不可以被接受的。


## 2 Funk SVD
直到 2006年 Netflix Prize 中 Simon Funk 在博客公开的算法。。它不在将矩阵分解为3个矩阵，而是分解为2个低秩的用户项目矩阵，在这里低秩的解释可以是：在大千世界中，总会存在相似的人或物，即物以类聚，人以群分。在这里，笔者总是混淆稀疏矩阵与低秩矩阵的概念，所以特此说明一下：  
稀疏矩阵（sparse matrix）：指的是矩阵中的非零元素比较少，但不一定是低秩的。比如对角矩阵，稀疏但是却满秩。  
低秩矩阵（low-rank matrix）：指的是矩阵的秩比较小，但不一定是稀疏的。比如全为1的矩阵，秩虽然小仅为1，但确实稠密矩阵。  
Simon Funk的思想很简单：可以直接通过训练集中的观察值利用最小化均方根学习P，Q矩阵。这种模型也被称作是 LFM （隐语义模型）  
$$
M_{m\times{n}}=P^T_{m\times{k}}Q_{k\times{n}}
$$
对于某一个用户的评分使用 Funk SVD 进行矩阵分解得到的结果就是：
$$
\hat{r_{ui}} = q^T_i{p_u}
$$
借鉴线性回归的思想，通过最小化观察数据的平方来寻求最优的用户和项目的隐含向量表示。同时为了避免过度拟合（Overfitting）观测数据，又提出了带有L2正则项的FunkSVD，需最小化的损失函数如下：
$$
\sum_{r_{ui}\in{R_{train}}}(r_{ui}-\hat{r_{ui}})^2+\lambda(||q_i||^2+||p_u||^2)
$$
在 Funk-SVD 获得巨大成功之后，很多著名的模型都是对 Funk-SVD 进行缝缝补补得到的（详情可参见 Netflix Prize `Koren:2009` `Ricci:2010`），于是就有了在预测模型中添加三项偏移的模型，被称为 BaisSVD。  
- Biased Item
- Biased User
- Biased Mean
Biased Item（物品偏移），表示了物品接受的评分和用户没有多大关系，物品本身质量决定了的偏移。  
Biased User（用户偏移），有些用户喜欢打高分，有些用户喜欢打低分，用户决定的偏移。  
Biased Mean（全局平均值偏移），根据全局打分设置的偏移，可能和整体用户群和物品质量有相对应的关系。  
加偏移后，对于某一个用户的评分使用 Funk SVD 进行矩阵分解得到的结果就是：
$$
\hat{r_{ui}} = q^T_i{p_u}+\mu+b_u+b_i
$$
加偏移后，需最小化的损失函数如下：
$$
\sum_{r_{ui}\in{R_{train}}}(r_{ui}-\hat{r_{ui}})^2+\lambda(||q_i||^2+||p_u||^2+||b_u||^2+||b_i||^2)\\
损失e_{ui}=r_{ui}-\hat{r_{ui}}
$$
使用随机梯度下降更新参数：
$$
b_u\longleftarrow{b_u+\gamma(e_{ui}-\lambda{b_u})}\\
b_i\longleftarrow{b_i+\gamma(e_{ui}-\lambda{b_i})}\\
q_i\longleftarrow{q_i+\gamma(e_{ui}p_u-\lambda{q_i})}\\
p_u\longleftarrow{p_u+\gamma(e_{ui}q_i-\lambda{p_u})}
$$
公式中$\gamma$表示学习率

In [1]:
from surprise import SVD, accuracy, Dataset
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size=.2, shuffle=True, random_state=10)

In [3]:
model = SVD(
    n_factors=100,#隐因子的大小，即矩阵分解的k值
    n_epochs=20,
    biased=True,#是否加偏移
    init_mean=0,#假定隐因子向量服从正态分布的均值。默认0
    init_std_dev=.1,#假定隐因子向量服从正态分布的方差。默认0.1
    lr_all=.005,#所有参数的学习率
    reg_all=.002,#所有参数的正则化惩罚参数
    lr_bi=None,#bi的学习率，优先于lr_all
    lr_bu=None,#bu的学习率，优先于lr_all
    lr_qi=None,#qi的学习率，优先于lr_all
    lr_pu=None,#pu的学习率，优先于lr_all
    reg_bi=None,#bi的正则化惩罚参数，优先于reg_all
    reg_bu=None,#bu的正则化惩罚参数，优先于reg_all
    reg_qi=None,#qi的正则化惩罚参数，优先于reg_all
    reg_pu=None,#pu的正则化惩罚参数，优先于reg_all
    random_state=10,
    verbose=True
)
model.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [4]:
pu = model.pu
pu.shape#m*k

(943, 100)

In [5]:
qi = model.qi
qi.shape#n*k

(1653, 100)

In [6]:
bu = model.bu
bu[:10]

array([-0.67255148, -0.08788255,  0.01496852, -0.31788629, -0.04793542,
       -0.33835399,  0.24522479, -0.10872228, -0.73829542, -0.22867225])

In [7]:
bi = model.bi
bi[:10]

array([ 0.93581937, -0.18170169,  0.29516143,  0.34899833, -0.8374666 ,
        0.25968677,  0.61021142,  0.59927492,  0.10486315,  0.34614277])

In [8]:
pred = model.test(testset)
pred[:5]

[Prediction(uid='154', iid='302', r_ui=4.0, est=4.364750739575204, details={'was_impossible': False}),
 Prediction(uid='896', iid='484', r_ui=4.0, est=3.585219432176975, details={'was_impossible': False}),
 Prediction(uid='230', iid='371', r_ui=4.0, est=3.276644594183164, details={'was_impossible': False}),
 Prediction(uid='234', iid='294', r_ui=3.0, est=2.4678726199793917, details={'was_impossible': False}),
 Prediction(uid='25', iid='729', r_ui=4.0, est=3.699794825875233, details={'was_impossible': False})]

In [9]:
trainset.to_inner_iid('302'), trainset.to_inner_uid('154')

(171, 738)

In [10]:
model.predict("154", "302")

Prediction(uid='154', iid='302', r_ui=None, est=4.364750739575204, details={'was_impossible': False})

In [11]:
accuracy.rmse(pred)

RMSE: 0.9453


0.9453314204573949

In [12]:
global_mean = trainset.global_mean
global_mean

3.5282375

In [13]:
#验证结果和pred第一行是否一致
import numpy as np
global_mean + bu[738] + bi[171] + np.dot(pu[738], qi[171])

4.364750739575204